<a href="https://colab.research.google.com/github/juliuserbach/Semantic-Features/blob/master/Mask_R_CNN_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/ftaubner/semantic_features_detection.git

fatal: destination path 'semantic_features_detection' already exists and is not an empty directory.


In [0]:
#!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%21100024&authkey=ADLJzehr2ENRggw" -O mapillary_vistas.zip

In [6]:
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%21101634&authkey=ABTiTiHVXMI06vU"  -O mapillary_prelim.h5

--2020-05-22 17:19:22--  https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%21101634&authkey=ABTiTiHVXMI06vU
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://mmhp0a.am.files.1drv.com/y4mY-K1IgzETm4YVAGoVFpRoGzhoiw0zfjpauVP0yUhYPSmk6eG4eg-T_5PF_M-J4tMVCUKSneD5aDgeFyPYjwv_mqm3hCVT5kICPtefvaMOL-j9w-Eg_qLoJZ2l6p0Er1aWzRBWppH_hrMztu-YKNxslBH0YpciW17lLJEBsgJLw5B3LMHUHFXf9v5m21Q6WJyw8MHGjZQaOM6HjY0WKusJQ/mask_rcnn_mapvistas_0083.h5?download&psid=1 [following]
--2020-05-22 17:19:23--  https://mmhp0a.am.files.1drv.com/y4mY-K1IgzETm4YVAGoVFpRoGzhoiw0zfjpauVP0yUhYPSmk6eG4eg-T_5PF_M-J4tMVCUKSneD5aDgeFyPYjwv_mqm3hCVT5kICPtefvaMOL-j9w-Eg_qLoJZ2l6p0Er1aWzRBWppH_hrMztu-YKNxslBH0YpciW17lLJEBsgJLw5B3LMHUHFXf9v5m21Q6WJyw8MHGjZQaOM6HjY0WKusJQ/mask_rcnn_mapvistas_0083.h5?download&psid=1
Resolving mmhp0a.am.files.1d

In [0]:
#!unzip -qq /content/mapillary_vistas.zip 'mapillary_vistas/validation/*' -d /content

In [0]:
#!unzip -qq /content/mapillary_vistas.zip 'mapillary_vistas/config.json' -d /content

In [9]:
!pip install imgaug
!pip install Cython
!pip install pycocotools

In [13]:
import os
os.chdir('/content/semantic_features_detection/samples/mapillary/cytools')
!python setup.py build_ext --inplace
os.chdir('/content/semantic_features_detection/Notebook')

running build_ext
building 'mask_tools' extension
creating build
creating build/temp.linux-x86_64-3.6
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -I/usr/include/python3.6m -c mask_tools.c -o build/temp.linux-x86_64-3.6/mask_tools.o
In file included from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1832:0,
                 from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from mask_tools.c:613:
/usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Usin

In [14]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import cv2
import imageio
%tensorflow_version 1.x
import tensorflow as tf


# Root directory of the project
ROOT_DIR = os.path.abspath("../")
# Ignore Warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
print(ROOT_DIR)
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/mapillary/"))  # To find local version
import mapillary_felix

%matplotlib inline

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
print(MODEL_DIR)

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")
print('finished')

/content/semantic_features_detection
/content/semantic_features_detection/logs
... done downloading pretrained model!
finished


In [0]:
class_names= ['Unlabeled', 'Bird', 'Ground Animal', 'Curb', 'Fence', 'Guard Rail', 
              'Barrier', 'Wall', 'Bike Lane', 'Crosswalk - Plain', 'Curb Cut', 'Parking', 
              'Pedestrian Area', 'Rail Track', 'Road', 'Service Lane', 'Sidewalk', 'Bridge', 
              'Building', 'Tunnel', 'Person', 'Bicyclist', 'Motorcyclist', 'Other Rider', 
              'Lane Marking - Crosswalk', 'Lane Marking - General', 'Mountain', 'Sand', 
              'Sky', 'Snow', 'Terrain', 'Vegetation', 'Water', 'Banner', 'Bench', 'Bike Rack',
              'Billboard', 'Catch Basin', 'CCTV Camera', 'Fire Hydrant', 'Junction Box', 'Mailbox',
              'Manhole', 'Phone Booth', 'Pothole', 'Street Light', 'Pole', 'Traffic Sign Frame', 
              'Utility Pole', 'Traffic Light', 'Traffic Sign (Back)', 'Traffic Sign (Front)',
              'Trash Can', 'Bicycle', 'Boat', 'Bus', 'Car', 'Caravan', 'Motorcycle', 'On Rails',
              'Other Vehicle', 'Trailer', 'Truck', 'Wheeled Slow', 'Car Mount', 'Ego Vehicle']
selected_classes = [34, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52]

print(selected_classes)

In [0]:
# TRAIN_DIR = '/content/mapillary_vistas'
# # Training dataset
# dataset_train = mapillary.MapillaryDataset()
# dataset_train.load_vistas(dataset_dir=TRAIN_DIR, subset='training', class_ids=selected_classes)
# dataset_train.prepare()

# Validation dataset
VAL_DIR = '/content/drive/My Drive/mapillary_vistas/mapillary_vistas'

dataset_val = mapillary.MapillaryDataset()
dataset_val.load_vistas(dataset_dir=VAL_DIR, subset='validation', class_ids=selected_classes)
dataset_val.prepare()

In [0]:
image_ids = np.random.choice(dataset_val.image_ids, 4)
for image_id in image_ids:
    image = dataset_val.load_image(image_id)
    print(dataset_val.class_names)
    mask, class_ids = dataset_val.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_val.class_names[1:])

In [0]:
class InferenceConfig(mapillary.mapvistas):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0
    NUM_CLASSES = len(selected_classes) + 1
config = InferenceConfig()
config.display()
batch_size = config.BATCH_SIZE

In [0]:
# Create model in training mode
model = modellib.MaskRCNN(mode="inference", config=config,
                          model_dir=MODEL_DIR)
model_path = '/content/mapillary_prelim.h5'
model.load_weights(model_path, by_name=True)


In [0]:
# def compute_batch_ap(image_ids):
#     APs = []
#     for image_id in image_ids:
#         # Load image
#         image, image_meta, gt_class_id, gt_bbox, gt_mask =\
#             modellib.load_image_gt(dataset_val, config,
#                                    image_id, use_mini_mask=False)
#         # Run object detection
#         results = model.detect([image], verbose=0)
#         # Compute AP
#         r = results[0]
#         AP, precisions, recalls, overlaps =\
#             utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
#                               r['rois'], r['class_ids'], r['scores'], r['masks'])
#         APs.append(AP)
#     return APs

In [0]:
# image_ids = np.random.choice(dataset_val.image_ids, 300)
# APs = compute_batch_ap(image_ids)
# print("mAP @ IoU=50: ", np.mean(APs))

In [0]:
def compute_batch_ap_classwise(image_ids):
    mAPs = np.zeros([len(selected_classes)])
    numAPs = np.zeros([len(selected_classes)])
    for image_id in image_ids:
        # Load image
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset_val, config,
                                  image_id, use_mini_mask=False)
        # Run object detection
        results = model.detect([image], verbose=0)
        # Compute AP
        r = results[0]
        # visualize.display_instances(image, boxes=r['rois'], masks=r['masks'], class_ids=r['class_ids'], class_names=dataset_val.class_names[1:], scores=r['scores'])
        # visualize.display_instances(image, boxes=gt_bbox, masks=gt_mask, class_ids=gt_class_id, class_names=dataset_val.class_names[1:])

        for i, class_id in enumerate(range(17)):
            # Select one class
            mask = np.where(gt_class_id == class_id, True, False)
            gt_class_id_sel = gt_class_id[mask]

            gt_bbox_sel = gt_bbox[mask,:]
            gt_mask_sel = gt_mask[:,:,mask]
                            
            # Select one class
            mask = np.where(r['class_ids'] == class_id, True, False)
            r_rois_sel = r['rois'][mask,:]
            r_class_ids_sel = r['class_ids'][mask]
            r_scores_sel = r['scores'][mask]
            r_masks_sel = r['masks'][:,:,mask] 
            if len(gt_class_id_sel) != 0:
                AP, precisions, recalls, overlaps =\
                    utils.compute_ap(gt_bbox_sel, gt_class_id_sel, gt_mask_sel,
                                      r_rois_sel, r_class_ids_sel, r_scores_sel, r_masks_sel)
                print("precisions: {}".format(precisions))
                mAPs[i] = mAPs[i] + AP
                # print("mAPs: {}".format(mAPs))
                numAPs[i] += 1
                # print("numAPs: {}".format(numAPs))

    # print("mAPs: {}".format(mAPs))
    # print("numAPs: {}".format(numAPs))
    mAPs = mAPs / numAPs
    return mAPs

In [0]:
image_ids = np.random.choice(dataset_val.image_ids,2)
# image_ids = [50]
APs = compute_batch_ap_classwise(image_ids)
print("mAP @ IoU=50: ", APs)

precisions: [1.  1.  0.5 0. ]
precisions: [1.  1.  0.5 0. ]
precisions: [0.5        0.5        0.5        0.5        0.5        0.4
 0.33333333 0.        ]
precisions: [0. 0.]
precisions: [0.33333333 0.33333333 0.33333333 0.33333333 0.25       0.2
 0.16666667 0.14285714 0.125      0.11111111 0.        ]
precisions: [1.         1.         0.5        0.33333333 0.25       0.2
 0.        ]
precisions: [1. 1. 0.]
precisions: [0. 0. 0.]
precisions: [0. 0. 0.]
precisions: [1. 1. 0.]
mAP @ IoU=50:  [       nan 0.0625            nan        nan        nan 0.
        nan        nan        nan        nan 0.25925926        nan
 0.06666667 0.16666667 0.         0.25       1.        ]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in true_divide


In [0]:
def compute_batch_ap_classwise_thresh(image_ids, class_id, thresh):
    mPs = 0
    numPs = 0
    for image_id in image_ids:
        # Load image
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset_val, config,
                                  image_id, use_mini_mask=False)
        # Run object detection
        results = model.detect([image], verbose=0)
        # Compute AP
        r = results[0]
        # visualize.display_instances(image, boxes=r['rois'], masks=r['masks'], class_ids=r['class_ids'], class_names=dataset_val.class_names[1:], scores=r['scores'])
        # visualize.display_instances(image, boxes=gt_bbox, masks=gt_mask, class_ids=gt_class_id, class_names=dataset_val.class_names[1:])

    
        # Select one class
        mask = np.where(gt_class_id == class_id, True, False)
        gt_class_id_sel = gt_class_id[mask]

        gt_bbox_sel = gt_bbox[mask,:]
        gt_mask_sel = gt_mask[:,:,mask]
                        
        # Select class and scores higher than threshold
        mask = np.where(np.logical_and(r['class_ids'] == class_id, r['scores'] > thresh), True, False)
        r_rois_sel = r['rois'][mask,:]
        r_class_ids_sel = r['class_ids'][mask]
        r_scores_sel = r['scores'][mask]
        r_masks_sel = r['masks'][:,:,mask] 
        if len(r_class_ids_sel) != 0:
            AP, precisions, recalls, overlaps =\
                utils.compute_ap(gt_bbox_sel, gt_class_id_sel, gt_mask_sel,
                                  r_rois_sel, r_class_ids_sel, r_scores_sel, r_masks_sel)
            mPs = mPs + precisions[-2]
            numPs += 1

    print("numPs: {}".format(numPs))
    mPs = mPs / numPs
    return mPs

In [0]:
mAP_opt = 0
thresh_opt = 0
class_id = 9
image_ids = np.random.choice(dataset_val.image_ids,500)

for thresh in range(20):
    thresh = thresh/20
    mAP = compute_batch_ap_classwise_thresh(image_ids, class_id, thresh)
    print(mAP)
    print(thresh)
    if mAP > mAP_opt:
        mAP_opt = mAP
        thresh_opt = thresh

/content/semantic_features_detection/mrcnn/utils.py:734: RuntimeWarning: invalid value encountered in true_divide
  recalls = np.cumsum(pred_match > -1).astype(np.float32) / len(gt_match)


In [0]:
print(mAP_opt)
print(thresh_opt)

0.09761124285445973
0.0


In [0]:
SAVE_DIR = '/content/Results'
image_ids = np.random.choice(dataset_val.image_ids,4)
for image_id in image_ids:
    image = dataset_val.load_image(image_id)
    results = model.detect([image], verbose=0)
    r = results[0]
    visualize.save_image(image = image[:,:,::-1], image_name=image_id, boxes=r['rois'], masks=r['masks'], class_ids=r['class_ids'], class_names=class_names, scores=r['scores'], save_dir=SAVE_DIR)

In [0]:
class_names = ['Bench', 'Billboard', 'Catch Basin', 'CCTV Camera', 'Fire Hydrant', 'Junction Box', 'Mailbox',
               'Manhole', 'Phone Booth', 'Street Light', 'Pole', 'Traffic Sign Frame', 'Utility Pole',
               'Traffic Light', 'Traffic Sign (Back)', 'Traffic Sign (Front)', 'Trash Can']
print(dataset_val.class_names[1:])

['Bench', 'Billboard', 'Catch Basin', 'CCTV Camera', 'Fire Hydrant', 'Junction Box', 'Mailbox', 'Manhole', 'Phone Booth', 'Street Light', 'Pole', 'Traffic Sign Frame', 'Utility Pole', 'Traffic Light', 'Traffic Sign (Back)', 'Traffic Sign (Front)', 'Trash Can']


In [0]:
for i in range(len(APs)):
    print("class name: {}: mAP: {}".format(class_names[i], APs[i]))


class name: Bench: mAP: nan
class name: Billboard: mAP: 0.13378898509652173
class name: Catch Basin: mAP: 0.08723088352212927
class name: CCTV Camera: mAP: 0.009009009009009009
class name: Fire Hydrant: mAP: 0.05945945945945946
class name: Junction Box: mAP: 0.018995633194279982
class name: Mailbox: mAP: 0.0
class name: Manhole: mAP: 0.1898932374137172
class name: Phone Booth: mAP: 0.0
class name: Street Light: mAP: 0.16076993966647124
class name: Pole: mAP: 0.07435044233169
class name: Traffic Sign Frame: mAP: 0.0
class name: Utility Pole: mAP: 0.10728754850566499
class name: Traffic Light: mAP: 0.22378939301032894
class name: Traffic Sign (Back): mAP: 0.06839771660148428
class name: Traffic Sign (Front): mAP: 0.24776525653562692
class name: Trash Can: mAP: 0.06402141058740836
